### Customer Churn Prediction for SyriaTel: A Binary Classification Approach

## 1. Business Understanding 

### 1.1 Problem Statement

SyriaTel is a mobile telecommunications company that has built an unquestionable reputation in the industry over the years. It has established a strong and consistent source of revenue, thanks to its unrivaled market dominance.

Like any other telecommunication business operating in a competitive environment, the telecommunication giant is taking precautionary measures to cushion itself against potential risks - particularly if the competition intensifies and some customers decide to switch loyalty to other emerging service providers. In addition, the company is interested in the factors that make customers switch loyalty, come up with customer retention policies, and maximize profits.

Therefore, as a senior data scientist of the telecommunication giant, I have been tasked to come up with a predictive machine learning model to evaluate whether a customer will churn SyriaTel or remain loyal to the company in the near future.

### 1.2 Business Objectives
* Come up with a model to predict whether a customer churns or stays.
* Analyze the factors contributing to churning.

### 1.3 Success Metrics


## 2. Data Understanding

In [7]:
# importing libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm 
import re

#import sklearn libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTEN
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score, roc_curve

import warnings 
warnings.filterwarnings("ignore");

In [8]:
# loading and reading the data 
Pro_Data = pd.read_csv("../Data/SyriaTel.csv")
Pro_Data.head() #reading the first five rows of the data

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [9]:
# reading the last five rows od the data
Pro_Data.tail()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False
3332,TN,74,415,400-4344,no,yes,25,234.4,113,39.85,...,82,22.60,241.4,77,10.86,13.7,4,3.70,0,False
